# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298396155299                   -0.85    5.1
  2   -8.300205499319       -2.74       -1.26    1.5
  3   -8.300448738339       -3.61       -1.89    2.8
  4   -8.300462605883       -4.86       -2.73    1.9
  5   -8.300464539054       -5.71       -3.23    2.6
  6   -8.300464608109       -7.16       -3.34    5.2
  7   -8.300464624560       -7.78       -3.47    9.1
  8   -8.300464634309       -8.01       -3.60    1.1
  9   -8.300464641303       -8.16       -3.79    1.0
 10   -8.300464642962       -8.78       -3.93    1.9
 11   -8.300464643937       -9.01       -4.39    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67442274730                   -0.70    6.4
  2   -16.67850804434       -2.39       -1.14    1.4
  3   -16.67923683832       -3.14       -1.86    4.0
  4   -16.67928283320       -4.34       -2.77    3.2
  5   -16.67928612102       -5.48       -3.21    5.9
  6   -16.67928621487       -7.03       -3.52    3.5
  7   -16.67928622325       -8.08       -3.98    1.0
  8   -16.67928622425       -9.00       -4.62    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32699626481                   -0.56    7.4
  2   -33.33426088147       -2.14       -1.00    1.2
  3   -33.33599950807       -2.76       -1.72    6.1
  4   -33.33685744260       -3.07       -2.33    6.5
  5   -33.33693832187       -4.09       -2.93    7.0
  6   -33.33694390069       -5.25       -3.85    3.6
  7   -33.33694392085       -7.70       -4.26    4.0


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298257133435                   -0.85    5.4
  2   -8.300239384070       -2.70       -1.59    1.0
  3   -8.300454595037       -3.67       -2.60    4.6
  4   -8.300403098979   +   -4.29       -2.36    9.2
  5   -8.300464310700       -4.21       -3.53    2.6
  6   -8.300464578997       -6.57       -3.90    2.8
  7   -8.300464643129       -7.19       -4.82    4.5


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32582676723                   -0.56    7.0
  2   -33.31450140746   +   -1.95       -1.27    1.0
  3   -15.19647434870   +    1.26       -0.44    8.5
  4   -33.32982127429        1.26       -1.90    6.5
  5   -33.18326080221   +   -0.83       -1.30    4.5
  6   -32.88407101914   +   -0.52       -1.24    4.6
  7   -33.31948718541       -0.36       -1.91    4.6
  8   -33.33624332091       -1.78       -2.39    3.4
  9   -33.33665720078       -3.38       -2.66    2.8
 10   -33.33680907228       -3.82       -2.71    2.8
 11   -33.33691713938       -3.97       -3.27    2.0
 12   -33.33694327200       -4.58       -3.78    3.2
 13   -33.33694365910       -6.41       -3.88    4.5
 14   -33.33694382332       -6.78       -4.18    2.5


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.